# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with 

'mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json'

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database 'uk_food' was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65da1ce72d3c657729d28fa3'),
 'establishments': {'RatingValue': None,
                    'geocode': {'latitude': None, 'longitude': None},
                    'latitude': None,
                    'longitude': None},
 'geocode': {'latitude':

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [9]:
# Insert the new restaurant into the collection
if establishments.find_one({"BusinessName":new_restaurant["BusinessName"]}) == None:
    establishments.insert_one(new_restaurant)
    print(f'Adding object with objectID: {establishments["BusinessName"]}')
              
    # Optional: Print a statement if the object is in the database. 
else: 
    print(f'Restaurant is already in the database.')

Restaurant is already in the database.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [10]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {'BusinessType' : "Restaurant/Cafe/Canteen"}
fields = {'BusinessTypeID': 1, 'BusinessType': 1 }


# Cast the results as a list and save them to a variable
results = list(establishments.find(query, fields))

# Pretty print the results
pprint(results)

[{'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65da1ce72d3c657729d28fac')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65da1ce72d3c657729d28fae')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65da1ce72d3c657729d28fb2')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65da1ce72d3c657729d28fb6')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65da1ce72d3c657729d28fb7')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65da1ce72d3c657729d28fc6')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65da1ce72d3c657729d28fcf')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('65da1ce72d3c657729d28fd0')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_

3. Update the new restaurant with the `BusinessTypeID` you found.

In [11]:
# Update the new restaurant with the correct BusinessTypeID

establishments.update_one( {"BusinessName": 'Penang Flavours'},
                             {'$set': {'BusinessTypeID': '1'}})
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}
pprint(establishments.find_one(query, projection=fields))

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65da1ce72d3c657729d28fac')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [12]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName':  "Dover"}

results = establishments.find(query)           
print("Number of documents that have LocalAuthorityName as Dover:", establishments.count_documents(query))




Number of documents that have LocalAuthorityName as Dover: 0


In [13]:
#Print all Douments with LocalAuthorityName is Dover
for result in results:
    print(result)

In [14]:
# Delete all documents where LocalAuthorityName is "Dover"

query = {'LocalAuthorityName':  "Dover"}
if establishments.find_one(query) == None:
    print("nothing to delete")
else: 
    establishments.delete_many(query)
    print("Deletion done for Dover")    


nothing to delete


In [15]:
# Check if any remaining documents include Dover
establishments.count_documents(query)

0

In [16]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('65da1ce72d3c657729d28fa3'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': 4,
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.196408, 'latitude': 51.086058},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCo

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [17]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({},[
                                {'$set': 
                                    { "geocode.latitude" : 
                                        {'$toDouble': "$geocode.latitude"}},
                                                                       
                                }
                                                                         
                            ]   
                        )
establishments.update_many({},[ { '$set':{"geocode.longitude": {'$toDouble':"$geocode.longitude"}} }])                                                       

In [18]:
#Print one document to see if the geocode latitude ,longitude has changed.
pprint(establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65da1ce72d3c657729d28fa3'),
 'establishments': {'RatingValue': None,
                    'geocode': {'latitude': None, 'longitude': None},
                    'latitude': None,
                    'longitude': None},
 'geocode': {'latitude':

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [20]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({},[
                                {'$set': 
                                    { "RatingValue" : 
                                        {'$toInt': "$RatingValue"}},
                                                                       
                                }
                                                                         
                            ]
                        )

In [21]:
#print a record to see rating value is also changed
pprint(establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65da1ce72d3c657729d28fa3'),
 'establishments': {'RatingValue': None,
                    'geocode': {'latitude': None, 'longitude': None},
                    'latitude': None,
                    'longitude': None},
 'geocode': {'latitude':

In [22]:
# Check that the coordinates and rating value are now numbers
check_doc = establishments.find_one()
for key,value in check_doc.items():
    print(f'{key}:{type(value)}')

_id:<class 'bson.objectid.ObjectId'>
FHRSID:<class 'int'>
ChangesByServerID:<class 'int'>
LocalAuthorityBusinessID:<class 'str'>
BusinessName:<class 'str'>
BusinessType:<class 'str'>
BusinessTypeID:<class 'int'>
AddressLine1:<class 'str'>
AddressLine2:<class 'str'>
AddressLine3:<class 'str'>
AddressLine4:<class 'str'>
PostCode:<class 'str'>
Phone:<class 'str'>
RatingValue:<class 'int'>
RatingKey:<class 'str'>
RatingDate:<class 'str'>
LocalAuthorityCode:<class 'str'>
LocalAuthorityName:<class 'str'>
LocalAuthorityWebSite:<class 'str'>
LocalAuthorityEmailAddress:<class 'str'>
scores:<class 'dict'>
SchemeType:<class 'str'>
geocode:<class 'dict'>
RightToReply:<class 'str'>
Distance:<class 'float'>
NewRatingPending:<class 'bool'>
meta:<class 'dict'>
links:<class 'list'>
establishments:<class 'dict'>
